In [6]:
%load_ext autoreload
%autoreload 1
%matplotlib inline

import os
import shutil
import sys
import subprocess

%aimport cnv_inference_config
project_config = cnv_inference_config
os.chdir(project_config.MB_ROOT)

from collections import defaultdict, OrderedDict
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

%aimport util
%aimport workspace_manager

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
workspace = {
    "scDNA" : workspace_manager.WorkspaceManager(
        task_name="ase_to_cnv",
        experiment_info={
            "data" : "scDNA",
            "sample" : "first_sample"
        },
        verbose=True
    ),
    "scRNA" : workspace_manager.WorkspaceManager(
        task_name="ase_to_cnv",
        experiment_info={
            "data" : "scRNA",
            "sample" : "first_sample"
        },
        verbose=True
    )
}

filtering_tag = "all"
clustering_tag = {
    "scDNA" : "cnv_3",
    "scRNA" : "seurat"
}
requirements = {
    "scDNA" : {
        "gene_counts" : f"{filtering_tag}_gene_counts.pkl",
        "clustering" : f"{clustering_tag['scDNA']}_clustering.pkl",
        "ase" : f"{filtering_tag}_{clustering_tag['scDNA']}_ase.pkl"
    },
    "scRNA" : {
        "gene_counts" : f"{filtering_tag}_gene_counts.pkl",
        "clustering" : f"{clustering_tag['scRNA']}_clustering.pkl",
        "ase" : f"{filtering_tag}_{clustering_tag['scRNA']}_ase.pkl"
    }
}

for data_type in ["scDNA", "scRNA"]:
    workspace[data_type].prepare_workspace(    
        requirements=requirements[data_type]
    )

processing read-only data directory
processing workspace directory
processing image directory
loading data to workspace directory


/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/all_gene_counts.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scDNA/gene_counts.pkl


/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/cnv_3_clustering.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scDNA/clustering.pkl
/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/all_cnv_3_ase.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scDNA/ase.pkl
processing read-only data directory
processing workspace directory
processing image directory
loading data to workspace directory


/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scRNA/all_gene_counts.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scRNA/gene_counts.pkl


/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scRNA/seurat_clustering.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scRNA/clustering.pkl
/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scRNA/all_seurat_ase.pkl —> /icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scRNA/ase.pkl


In [3]:
workspace["scDNA"]

Task name: ase_to_cnv
Experiment info:
{
    "data": "scDNA",
    "sample": "first_sample"
}
Config generated on 2019-08-13 17:26
Contents:
{
    "task_name": "ase_to_cnv",
    "experiment_info": {
        "data": "scDNA",
        "sample": "first_sample"
    },
    "cookiecutter_info": {
        "root": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference",
        "lib": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/lib",
        "notebooks": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/notebooks",
        "src": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/src",
        "img": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/img",
        "data": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data",
        "raw": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/raw",
        "tmp": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp",
        "processed": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed"
    },
    "config_dir": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/notebooks/ase_to_cnv",
    "data_dir": {
        "root": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed",
        "sample": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA"
    },
    "data": {
        "gene_counts": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/PC1_PC2_span_75%_gene_counts.pkl",
        "clustering": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/evo_dist_9_clustering.pkl",
        "ase": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scDNA/PC1_PC2_span_75%_evo_dist_9_ase.pkl"
    },
    "tmp_data_dir": {
        "root": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv",
        "sample": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scDNA"
    },
    "tmp_data": {
        "gene_counts": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scDNA/gene_counts.pkl",
        "clustering": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scDNA/clustering.pkl",
        "ase": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scDNA/ase.pkl"
    },
    "img_dir": {
        "root": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/img/ase_to_cnv",
        "sample": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/img/ase_to_cnv/first_sample"
    }
}

In [4]:
workspace["scRNA"]

Task name: ase_to_cnv
Experiment info:
{
    "data": "scRNA",
    "sample": "first_sample"
}
Config generated on 2019-08-13 17:26
Contents:
{
    "task_name": "ase_to_cnv",
    "experiment_info": {
        "data": "scRNA",
        "sample": "first_sample"
    },
    "cookiecutter_info": {
        "root": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference",
        "lib": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/lib",
        "notebooks": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/notebooks",
        "src": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/src",
        "img": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/img",
        "data": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data",
        "raw": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/raw",
        "tmp": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp",
        "processed": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed"
    },
    "config_dir": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/notebooks/ase_to_cnv",
    "data_dir": {
        "root": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed",
        "sample": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scRNA"
    },
    "data": {
        "gene_counts": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scRNA/PC1_PC2_span_75%_gene_counts.pkl",
        "clustering": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scRNA/seurat_clustering.pkl",
        "ase": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/processed/first_sample/scRNA/seurat_ase.pkl"
    },
    "tmp_data_dir": {
        "root": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv",
        "sample": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scRNA"
    },
    "tmp_data": {
        "gene_counts": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scRNA/gene_counts.pkl",
        "clustering": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scRNA/clustering.pkl",
        "ase": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/data/tmp/ase_to_cnv/first_sample/scRNA/ase.pkl"
    },
    "img_dir": {
        "root": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/img/ase_to_cnv",
        "sample": "/icgc/dkfzlsdf/analysis/B260/users/v390v/cnv_inference/img/ase_to_cnv/first_sample"
    }
}